In [165]:
import pandas as pd

from matplotlib import pyplot as plt

from sklearn.preprocessing import OneHotEncoder

from sklearn.linear_model import LinearRegression

import warnings

import csv



warnings.filterwarnings('ignore')

pd.plotting.register_matplotlib_converters()

In [166]:
df = pd.read_csv('data2.csv', parse_dates=['Timestamp'], encoding = 'utf-16')

df.head()

,Origin,Timestamp,Content,Title,Comment_count,Retweet_count,Polarity,Subjectivity
0,twitter,1552760166,Mooi verhaal over windmolens https://www.insta...,Null,1.0,0.0,0.70,1.00
1,twitter,1552995709,Het mooiste happy hour was afgelopen zaterdag....,Null,0.0,0.0,0.00,0.00
2,twitter,1553001655,Dus bij jou in de achtertuin windmolensvoortui...,Null,0.0,0.0,0.00,0.00
3,twitter,1552471319,Hoe kan provincie zorgen voor bouw van genoeg ...,Null,0.0,0.0,0.20,0.50
4,twitter,1552993305,We zijn geen communistisch land. Een vliegveld...,Null,0.0,0.0,0.19,0.58


In [167]:
df['Timestamp']= pd.to_datetime(df['Timestamp'], unit='s')

df.head()

,Origin,Timestamp,Content,Title,Comment_count,Retweet_count,Polarity,Subjectivity
0,twitter,2019-03-16 18:16:06,Mooi verhaal over windmolens https://www.insta...,Null,1.0,0.0,0.70,1.00
1,twitter,2019-03-19 11:41:49,Het mooiste happy hour was afgelopen zaterdag....,Null,0.0,0.0,0.00,0.00
2,twitter,2019-03-19 13:20:55,Dus bij jou in de achtertuin windmolensvoortui...,Null,0.0,0.0,0.00,0.00
3,twitter,2019-03-13 10:01:59,Hoe kan provincie zorgen voor bouw van genoeg ...,Null,0.0,0.0,0.20,0.50
4,twitter,2019-03-19 11:01:45,We zijn geen communistisch land. Een vliegveld...,Null,0.0,0.0,0.19,0.58


In [168]:
# load data


3


# remove outlier record

df = df.loc[df['Timestamp'] > '2007-01-01']

# FEATURE EXTRACTION

# month, for seasonality
df['month'] = 0
df['month'] = df['Timestamp'].dt.month

# days_since_first, for trend
df['days_since_first'] = 0
df['days_since_first'] = (df['Timestamp'] - min(df['Timestamp'])).dt.days



# create test design

train_filter = df['Timestamp'] < '2017-08-01'

df_train = df[train_filter]

df_test = df[~train_filter]

df_train.head()

,Origin,Timestamp,Content,Title,Comment_count,Retweet_count,Polarity,Subjectivity,month,days_since_first
729,twitter,2017-07-28 09:39:02,34 windmolens verrijzen op en rond de Krammers...,Null,0.0,7.0,0.00,0.0000,7,3688
730,twitter,2017-07-27 10:23:31,Er komt een bezoekerscentrum met museum op de ...,Null,0.0,10.0,0.00,0.0000,7,3687
731,twitter,2017-07-26 19:31:17,Toch jammer dat lokale overheden regelmatig bo...,Null,3.0,21.0,-0.15,0.4625,7,3687
732,twitter,2017-07-25 21:58:52,Je bent wel historisch dom als je hedendaagse ...,Null,1.0,8.0,-0.25,0.7000,7,3686
733,twitter,2017-07-25 18:44:40,Afgezien daarvan waar moet de stroom vandaan k...,Null,1.0,10.0,0.10,0.3000,7,3686


In [169]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2569 entries, 0 to 2732
Data columns (total 10 columns):
Origin              2569 non-null object
Timestamp           2569 non-null datetime64[ns]
Content             2569 non-null object
Title               2569 non-null object
Comment_count       2569 non-null float64
Retweet_count       2569 non-null float64
Polarity            2569 non-null float64
Subjectivity        2569 non-null float64
month               2569 non-null int64
days_since_first    2569 non-null int64
dtypes: datetime64[ns](1), float64(4), int64(2), object(3)
memory usage: 220.8+ KB


In [170]:
print(df_train.count(), df_test.count())

Origin              1888
Timestamp           1888
Content             1888
Title               1888
Comment_count       1888
Retweet_count       1888
Polarity            1888
Subjectivity        1888
month               1888
days_since_first    1888
dtype: int64 Origin              681
Timestamp           681
Content             681
Title               681
Comment_count       681
Retweet_count       681
Polarity            681
Subjectivity        681
month               681
days_since_first    681
dtype: int64


In [171]:
# select features

# one-hot-encode month, to let the linear model recognize monthly seasonality

# features = ['month']

# ohe = OneHotEncoder(categorical_features=[0])

# x_train = ohe.fit_transform(df_train[features].astype(int))

# x_test = ohe.transform(df_test[features].astype(int))



# define targets

target = 'Polarity'

y_train = df_train[target]

y_test = df_test[target]



# # fit and aply linear regression model
import numpy as np

x_train = np.column_stack((df_train['month'], df_train['Subjectivity'])).astype(int)
x_test = np.column_stack((df_test['month'], df_test['Subjectivity'])).astype(int)
y_test = df_test[target]

model = LinearRegression()

model.fit(x_train, y_train)

pred_train = model.predict(x_train)

pred_test = model.predict(x_test)

df_test['predictions'] = pred_test

df_test.head()

,Origin,Timestamp,Content,Title,Comment_count,Retweet_count,Polarity,Subjectivity,month,days_since_first,predictions
0,twitter,2019-03-16 18:16:06,Mooi verhaal over windmolens https://www.insta...,Null,1.0,0.0,0.70,1.00,3,4285,0.441848
1,twitter,2019-03-19 11:41:49,Het mooiste happy hour was afgelopen zaterdag....,Null,0.0,0.0,0.00,0.00,3,4287,0.043928
2,twitter,2019-03-19 13:20:55,Dus bij jou in de achtertuin windmolensvoortui...,Null,0.0,0.0,0.00,0.00,3,4288,0.043928
3,twitter,2019-03-13 10:01:59,Hoe kan provincie zorgen voor bouw van genoeg ...,Null,0.0,0.0,0.20,0.50,3,4281,0.043928
4,twitter,2019-03-19 11:01:45,We zijn geen communistisch land. Een vliegveld...,Null,0.0,0.0,0.19,0.58,3,4287,0.043928


In [172]:
df.corr()

,Comment_count,Retweet_count,Polarity,Subjectivity,month,days_since_first
Comment_count,1.000000,0.647040,-0.002840,0.072814,0.014166,0.171918
Retweet_count,0.647040,1.000000,0.003935,0.048564,0.005847,0.195411
Polarity,-0.002840,0.003935,1.000000,0.228086,0.019913,0.014080
Subjectivity,0.072814,0.048564,0.228086,1.000000,-0.001238,0.154934
month,0.014166,0.005847,0.019913,-0.001238,1.000000,-0.117588
days_since_first,0.171918,0.195411,0.014080,0.154934,-0.117588,1.000000


In [ ]:
# plt.plot(df_train['Timestamp'], pred_train, label='train predictions', linestyle='--', color='blue')

# plt.plot(df_train['Timestamp'], y_train, label='train actuals', linestyle='-', color='blue')


plt.plot(df_test['Timestamp'], y_test, label='test actuals', linestyle='-', color='red')

plt.plot(df_test['Timestamp'], pred_test, label='test predictions', linestyle='--', color='blue')

plt.legend(loc='upper left', bbox_to_anchor=(1,1))

plt.xticks(rotation=30)

plt.ylabel(target)

plt.show()


In [174]:
from sklearn.metrics import r2_score
model.coef_

r2_score(df_test['Polarity'], pred_test) 

0.06241285625763793